In [42]:
import rospy
import time 
import numpy as np
import gym

from sensor_msgs.msg import Joy, Image
from webots import WebotsWorld
from webots.krock import Krock
from os import path
from gym import spaces

In [2]:
rospy.init_node("traversability_simulation")

In [3]:
w = WebotsWorld.from_file(path.abspath('/home/francesco/Documents/Master-Thesis/core/webots/krock/krock2_ros/worlds/krock2_camera.wbt'))

w()

/home/francesco/Documents/Master-Thesis/core/webots/krock/krock2_ros/worlds/krock2_camera.wbt


[WARN] [1541795353.079798]: /krock/supervisor/get_self transport error completing service call: unable to receive data from sender, check sender's logs for details


In [4]:
k = Krock(rate=rospy.Rate(hz=10))

In [5]:
k(w)

In [51]:
actions = spaces.Dict({ 
    'frontal_freq': spaces.Box(low=-1.0, high=1.0, shape=(), dtype=np.float),
    'lateral_freq': spaces.Box(low=-1.0, high=1.0, shape=(), dtype=np.float) 
})

actions.sample()

Dict(frontal_freq:Box(), lateral_freq:Box())


OrderedDict([('frontal_freq', array(0.20552675)),
             ('lateral_freq', array(0.08976637))])

In [60]:
observation_space = spaces.Dict({
     'sensors':  spaces.Dict({
            'position': spaces.Box(low=-5, high=5, shape=(3,)),
            'velocity': spaces.Box(low=-1, high=1, shape=(3,)),
            'front_cam': spaces.Tuple((
                spaces.Box(low=0, high=1, shape=(600, 800, 3)),
                spaces.Box(low=0, high=1, shape=(600, 800, 3))
            )),
    })
})

observation_space

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.


Dict(sensors:Dict(front_cam:Tuple(Box(600, 800, 3), Box(600, 800, 3)), position:Box(3,), velocity:Box(3,)))

In [39]:
k.move(gait=1,
                  frontal_freq=-2,
                  lateral_freq=0,
                  manual_mode=True)

In [55]:
k.stop()

shutdown request: new node registered with same name


In [54]:
for _ in range(100):
    random_actions = actions.sample()
    k.move(gait=1,
           frontal_freq=random_actions['frontal_freq'], 
           lateral_freq=random_actions['lateral_freq'], 
           manual_mode=True)
    k.sleep()

In [ ]:
d = np.zeros(3)

p = k.state['pose'].pose.position

d += [p.x, p.y, p.z]

for i in range(100):
    k.move(1,1,0)
    k.sleep()
    k.stop()
    
    p = k.state['pose'].pose.position

d = [p.x, p.y, p.z] - d
    
print(d)